In [1]:
print('hi')

hi


In [2]:
import os
import argparse
#import cv2
import sys
import numpy as np
#import insightface
from tqdm import tqdm
#from insightface.app import FaceAnalysis 
import pickle
import time
from leech import Leech
import sklearn
import math
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


#app = FaceAnalysis(providers=['CPUExecutionProvider'])

#app.prepare(ctx_id=0)

#feret
#def get_name(s):
#    return s[:5] 

#lfw
def get_name(s):
    return s[:-9] 


def get_names_vecs(folder):

    cache_file = 'embedding_'+folder
    if os.path.exists(cache_file):
        print('load from cache')
        data = pickle.load(open(cache_file,'rb'))
        names, vecs = data
        return names, vecs 
    
    names = []
    vecs = []
    name_list=[]
    cnt=0
    for f in os.listdir(folder):
        if f[-3:]=='jpg':
            name_list.append(f)

    for name in tqdm(name_list):
        img = cv2.imread(folder +'/'+ name)
        #print(folder+name)
        faces = app.get(img)
        if len(faces)!=1:
            continue
        
        img_embedding = faces[0].normed_embedding
        names.append(get_name(name))
        vecs.append(img_embedding)
    
    data = (names, vecs)
    pickle.dump(data,open(cache_file,'wb'))
    
    return names,vecs

names, vecs = get_names_vecs('lfw')

load from cache


In [3]:
import random
import numpy as np
random.seed(12345)

In [4]:
alpha = 2**20

def round(v):
    return [int(x*alpha) for x in v]

def padding(v):
    v=list(v.copy())
    while len(v)%24!=0:
        v.append(0)
    return np.array(v)
    
def decode_h(v,t):

    
    v=padding(v)
    
    v=round(v)

    ds = []
    os = []
    for i in range(len(v)//24):
        sub_v = v[i*24:i*24+24]
        decoded = Leech.decode(sub_v,t*alpha)
        offset = np.array(sub_v) - np.array(decoded)

        ds.append(decoded)
        os.append(offset)
    return np.concatenate(ds), np.concatenate(os)
    
    

In [5]:
from multiprocessing import Pool

def work(vec):
    p,o = decode_h(vec ,0.05)
    return p,o

reset = False

if not reset and os.path.exists('decode_cache'):
    print('load from cache')
    r = pickle.load(open('decode_cache','rb'))
else:
    with Pool(64) as p:
        r = list(tqdm(p.imap(work, vecs), total=len(vecs)))   
    pickle.dump(r,open('decode_cache','wb'))

100%|██████████| 10616/10616 [04:03<00:00, 43.61it/s]


In [46]:
from sklearn.feature_selection import r_regression

In [47]:
all_reg = []
for d in tqdm(range(512)):
    reg=r_regression(Ps, np.array(Os)[:,d])
    all_reg.extend(reg)
    
print(np.sqrt(np.square(all_reg).mean()))

100%|██████████| 512/512 [00:22<00:00, 22.52it/s]

0.0282459457551304
